In [1]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
import numpy as np

In [2]:
grid = gpd.read_file("data/seoul_500_grid.shp", encoding='euc-kr')

In [3]:
df = pd.read_csv("data/전국소방용수시설표준데이터.csv", encoding='cp949')

In [4]:
grid.head()

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((935525.546 1950335.888, 935511..."
1,11,Seoul,서울특별시,2,"POLYGON ((935290.584 1950669.394, 935290.011 1..."
2,11,Seoul,서울특별시,3,"POLYGON ((935725.680 1950165.230, 935716.358 1..."
3,11,Seoul,서울특별시,4,"POLYGON ((936034.844 1950175.305, 936025.248 1..."
4,11,Seoul,서울특별시,5,"POLYGON ((935554.925 1951125.083, 935561.312 1..."


In [5]:
df = df.drop(columns=['Unnamed: 13', 'Unnamed: 14',	'Unnamed: 15', 'Unnamed: 16'])

In [6]:
df = df[df['시도명'] == '서울특별시']

In [7]:
df

,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,상세위치,안전센터명,보호틀유무,설치연도,출수압력,관할기관명
4502,서울특별시,영등포구,11560,서울특별시 영등포구 선유로9나길 8,NaN,37.517146,126.886298,NaN,현장대응단,N,1900,3.0,영등포소방서
6904,서울특별시,영등포구,11560,서울특별시 영등포구 도림로125길 28-2,NaN,37.513013,126.893554,NaN,현장대응단,N,1900,2.5,영등포소방서
8590,서울특별시,종로구,11110,서울특별시 종로구 우정국로 67,서울특별시 종로구 견지동 13,37.575013,126.982465,불교신문,현장대응단,N,1964,NaN,종로소방서
8591,서울특별시,종로구,11110,서울특별시 종로구 우정국로 57,서울특별시 종로구 견지동 38-2,37.574122,126.982727,조계사 정문,현장대응단,N,1962,NaN,종로소방서
8592,서울특별시,종로구,11110,서울특별시 종로구 우정국로 49,서울특별시 종로구 견지동 55-2,37.573431,126.982746,신앙불교사 건물 ?x편,현장대응단,N,2001,NaN,종로소방서
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,서울특별시,관악구,11620,서울특별시 관악구 승방3길 36,서울특별시 관악구 남현동 602-325,37.472156,126.981746,NaN,현장대응단,N,2012,3.0,관악소방서
49996,서울특별시,관악구,11620,서울특별시 관악구 승방1길 32,서울특별시 관악구 남현동 602-230,37.472713,126.981466,NaN,현장대응단,N,1996,2.5,관악소방서
49997,서울특별시,관악구,11620,서울특별시 관악구 승방3나길 11,서울특별시 관악구 남현동 602-203,37.472661,126.980664,NaN,현장대응단,N,2004,3.0,관악소방서
49998,서울특별시,관악구,11620,서울특별시 관악구 승방7길 11,서울특별시 관악구 남현동 602-192,37.472763,126.979787,NaN,현장대응단,N,1992,2.5,관악소방서


In [8]:
grid.rename(columns={'EMD_CD': '읍면동코드', 'EMD_KOR_NM': '읍면동 이름'}, inplace=True)

In [9]:
grid.head()

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((935525.546 1950335.888, 935511..."
1,11,Seoul,서울특별시,2,"POLYGON ((935290.584 1950669.394, 935290.011 1..."
2,11,Seoul,서울특별시,3,"POLYGON ((935725.680 1950165.230, 935716.358 1..."
3,11,Seoul,서울특별시,4,"POLYGON ((936034.844 1950175.305, 936025.248 1..."
4,11,Seoul,서울특별시,5,"POLYGON ((935554.925 1951125.083, 935561.312 1..."


In [10]:
sigungu_mapping = {
    '11110': '종로구',
    '11140': '중구',
    '11170': '용산구',
    '11200': '성동구',
    '11215': '광진구',
    '11230': '동대문구',
    '11260': '중랑구',
    '11290': '성북구',
    '11305': '강북구',
    '11320': '도봉구',
    '11350': '노원구',
    '11380': '은평구',
    '11410': '서대문구',
    '11440': '마포구',
    '11470': '양천구',
    '11500': '강서구',
    '11530': '구로구',
    '11545': '금천구',
    '11560': '영등포구',
    '11590': '동작구',
    '11620': '관악구',
    '11650': '서초구',
    '11680': '강남구',
    '11710': '송파구',
    '11740': '강동구'
}

# 시군구코드 열 생성 (읍면동코드의 앞 5자리)
grid['시군구코드'] = grid['읍면동코드'].astype(str).str[:5]

# 시군구 이름 매핑
grid['시군구명'] = grid['시군구코드'].map(sigungu_mapping)

KeyError: '읍면동코드'

In [11]:
grid.head(3)

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((935525.546 1950335.888, 935511..."
1,11,Seoul,서울특별시,2,"POLYGON ((935290.584 1950669.394, 935290.011 1..."
2,11,Seoul,서울특별시,3,"POLYGON ((935725.680 1950165.230, 935716.358 1..."


In [12]:
grid = grid.to_crs(epsg=4326)

In [13]:
grid.head(3)

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((126.77009 37.55011, 126.76992 ..."
1,11,Seoul,서울특별시,2,"POLYGON ((126.76740 37.55310, 126.76739 37.553..."
2,11,Seoul,서울특별시,3,"POLYGON ((126.77237 37.54859, 126.77226 37.548..."


In [14]:
# df의 위도, 경도를 기반으로 gdf_points 생성
gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.경도, df.위도))

In [15]:
gdf_points.head(1)

,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,상세위치,안전센터명,보호틀유무,설치연도,출수압력,관할기관명,geometry
4502,서울특별시,영등포구,11560,서울특별시 영등포구 선유로9나길 8,NaN,37.517146,126.886298,NaN,현장대응단,N,1900,3.0,영등포소방서,POINT (126.88630 37.51715)


In [16]:
joined = gpd.sjoin(grid, gdf_points, how="left", op="contains")

C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\1qlqj\AppData\Local\Temp\ipykernel_34832\297099991.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  joined = gpd.sjoin(grid, gdf_points, how="left", op="contains")


In [17]:
grid['소방용수_수'] = joined.groupby('id').size()

In [18]:
grid.head(3)

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry,소방용수_수
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((126.77009 37.55011, 126.76992 ...",NaN
1,11,Seoul,서울특별시,2,"POLYGON ((126.76740 37.55310, 126.76739 37.553...",1.0
2,11,Seoul,서울특별시,3,"POLYGON ((126.77237 37.54859, 126.77226 37.548...",1.0


In [19]:
pd.set_option('display.max_rows', None)
grid['소방용수_수'].value_counts().sort_index()

소방용수_수
1.0      1310
2.0        47
3.0        59
4.0        36
5.0        41
6.0        33
7.0        30
8.0        34
9.0        34
10.0       27
11.0       19
12.0       31
13.0       20
14.0       16
15.0       12
16.0       28
17.0       15
18.0       22
19.0       19
20.0       21
21.0       17
22.0       10
23.0       17
24.0       17
25.0       19
26.0       20
27.0       18
28.0       18
29.0       14
30.0       15
31.0       25
32.0       22
33.0       19
34.0       12
35.0       16
36.0       15
37.0       14
38.0       17
39.0       13
40.0       19
41.0       16
42.0       16
43.0       17
44.0       11
45.0       13
46.0       14
47.0       12
48.0       12
49.0       16
50.0       18
51.0       16
52.0       11
53.0       13
54.0       17
55.0       16
56.0       14
57.0       11
58.0       12
59.0        8
60.0        7
61.0       10
62.0       10
63.0        7
64.0        9
65.0        8
66.0        3
67.0        5
68.0        9
69.0        6
70.0       11
71.0        4

매우 큰 이상치 값들이 존재한다. 그러나 상업지구, 공업지구, 다중이용시설이 밀집한 지역에서는, 소방용수 시설이 평균보다 훨씬 높을 수 있을 것이다.

In [20]:
# '소방용수_수' 열을 조건에 따라 변환하는 함수
def categorize_fire_water_count(x):
    if x >= 30:
        return '30'
    elif x >= 20:
        return '20'
    elif x >= 10:
        return '10'
    else:
        return x

# '소방용수_수' 열에 함수 적용
grid['소방용수_수'] = grid['소방용수_수'].apply(categorize_fire_water_count)

In [21]:
grid['소방용수_수'].value_counts()

소방용수_수
1.0    1310
30      623
10      209
20      171
3.0      59
2.0      47
5.0      41
4.0      36
9.0      34
8.0      34
6.0      33
7.0      30
Name: count, dtype: int64

In [22]:
grid.head()

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry,소방용수_수
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((126.77009 37.55011, 126.76992 ...",NaN
1,11,Seoul,서울특별시,2,"POLYGON ((126.76740 37.55310, 126.76739 37.553...",1.0
2,11,Seoul,서울특별시,3,"POLYGON ((126.77237 37.54859, 126.77226 37.548...",1.0
3,11,Seoul,서울특별시,4,"POLYGON ((126.77587 37.54870, 126.77576 37.548...",1.0
4,11,Seoul,서울특별시,5,"POLYGON ((126.77035 37.55722, 126.77042 37.557...",1.0


In [23]:
df = pd.DataFrame(grid)

In [24]:
df.head()

,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM,id,geometry,소방용수_수
0,11,Seoul,서울특별시,1,"MULTIPOLYGON (((126.77009 37.55011, 126.76992 ...",NaN
1,11,Seoul,서울특별시,2,"POLYGON ((126.76740 37.55310, 126.76739 37.553...",1.0
2,11,Seoul,서울특별시,3,"POLYGON ((126.77237 37.54859, 126.77226 37.548...",1.0
3,11,Seoul,서울특별시,4,"POLYGON ((126.77587 37.54870, 126.77576 37.548...",1.0
4,11,Seoul,서울특별시,5,"POLYGON ((126.77035 37.55722, 126.77042 37.557...",1.0


In [26]:
df = df.drop(columns=['CTPRVN_CD', 'CTP_ENG_NM', 'CTP_KOR_NM'])

In [27]:
df.head()

,id,geometry,소방용수_수
0,1,"MULTIPOLYGON (((126.77009 37.55011, 126.76992 ...",NaN
1,2,"POLYGON ((126.76740 37.55310, 126.76739 37.553...",1.0
2,3,"POLYGON ((126.77237 37.54859, 126.77226 37.548...",1.0
3,4,"POLYGON ((126.77587 37.54870, 126.77576 37.548...",1.0
4,5,"POLYGON ((126.77035 37.55722, 126.77042 37.557...",1.0


In [28]:
df.to_csv('data/seoul_500_grid_water.csv', encoding='euc-kr', index=False)